In [1]:
import pandas as pd
import numpy as np

eps = 10**-8

In [2]:
# train = pd.read_csv("output/sorted_train0310.csv")
# test = pd.read_csv("output/sorted_test0310.csv")
# val = pd.read_csv("output/sorted_val0310.csv")

In [3]:
train = pd.read_csv("output/random_train0310.csv")
test = pd.read_csv("output/random_test0310.csv")
val = pd.read_csv("output/random_val0310.csv")

train = train[train.LoanStatus != "MISSING"]
test = test[test.LoanStatus != "MISSING"]
val = val[val.LoanStatus != "MISSING"]

print train.head()[train.columns[:6]]

     BorrCity BorrState BorrZip        CDC_City CDC_State  CDC_Zip
0    SAN JOSE        CA   95111   San Francisco        CA  94133.0
1   KENNEWICK        WA   99336  Spokane Valley        WA  99212.0
2    KIRKLAND        WA   98033         Seattle        WA  98168.0
3    STAMFORD        CT    6902         Danbury        CT   6810.0
4  WALKERTOWN        NC   27051    Kernersville        NC  27284.0


In [4]:
# Compute some statistics
print 'Train set:'
chargeoff_count = (train.LoanStatus == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/m_train

print '\n', 'Test set:'
chargeoff_count = (test.LoanStatus == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/m_test

print '\n', 'Validation set:'
chargeoff_count = (val.LoanStatus == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/m_val

Train set:
Chargeoffs: 7124
Percentage of loans that defaulted:

NameError: name 'm_train' is not defined

In [ ]:
m_train = float(len(train))
m_test = float(len(test))
m_val = float(len(val))
print m_train, m_test, m_val

In [ ]:
# filtering for the columns you don't want to include for your model
c = train.columns.tolist()
c.remove('LoanStatus')
c.remove('ChargeOffDate')
c.remove('GrossChargeOffAmount')
c.remove('BorrZip')
c.remove('CDC_Zip')
c.remove('BorrCity')
c.remove('CDC_City')
c.remove('ThirdPartyLender_City')
# c.remove('BorrState')
# c.remove('CDC_State')
# c.remove('ThirdPartyLender_State')
c.remove('ProjectCounty')
c.remove('ProjectState')
c.remove('ApprovalDate')
print c

In [ ]:
x_train = train[c]
x_test = test[c]
x_val = val[c]

y_train_logit = train['LoanStatus'].values == "CHGOFF"
y_test_logit = test['LoanStatus'].values == "CHGOFF"
y_val_logit = val['LoanStatus'].values == "CHGOFF"

print x_train.head()[x_train.columns[:4]]
print '\n', y_train_logit[:5]

# hazard_target = train['ChargeOffDate']
# amount_target = train['GrossChargeOffAmount']

# print train.dtypes
# convert categorical variables to dummy variables
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
x_val = pd.get_dummies(x_val)

In [ ]:
# to get consistent feature dimensions for both train and test dataset
def add_missing_dummy_columns(d1, d2):
    missing_cols = set(d1.columns) - set(d2.columns)
    for c in missing_cols:
        d2[c] = 0
    return d2

print 'before fixing columns: '
print x_train.shape
print x_test.shape
print x_val.shape

def fix_columns(x_train, x_test):  

    x_test = add_missing_dummy_columns(x_train, x_test)

    extra_cols = set(x_test.columns) - set(x_train.columns)
    x_test = x_test[x_train.columns]
    return x_test

x_test = fix_columns(x_train, x_test)
x_val = fix_columns(x_train, x_val)
print 'after fixing columns: '
print x_train.shape
print x_test.shape
print x_val.shape

In [ ]:
# fit to models 
# below is to fit to logit model

# y_train_logit = np.reshape(l, (len(l),))
from sklearn.linear_model import LogisticRegression
solver = 'liblinear'

lr = LogisticRegression(solver=solver, 
                        multi_class= 'ovr', 
                        C=50.,
                        penalty = 'l2',
                        fit_intercept=True,
                        max_iter=200,
                        random_state=None, 
                       )
lr.fit(x_train, y_train_logit)

In [ ]:
y_probs_val = lr.predict_proba(x_val)
print y_probs_val[:5]

In [ ]:
y_preds_val = y_probs_val[:,1] > 0.2
print y_preds_val[:5].reshape(5,1)
print "\n", "validation accuracy", np.sum(y_preds_val == y_val_logit, dtype=float) / m_val
print "false positive rate",  np.sum((y_val_logit == 0) * (y_preds_val == 1)) / (np.sum(y_val_logit == 0, dtype=float) + eps)
print "true positive rate", np.sum((y_val_logit == 1) * (y_preds_val == 1)) / (np.sum(y_val_logit == 1, dtype=float) + eps)

In [ ]:
import matplotlib.pyplot as plt
fprs = []
tprs = []
step_size = 0.0001
for threshold in np.arange(0.001,1,step_size):
    y_preds_val = y_probs_val[:,1] > threshold
    num_pos = np.sum(y_preds_val == 1, dtype=float) + eps
    fprs.append(np.sum((y_val_logit == 0) * (y_preds_val == 1)) / (np.sum(y_val_logit == 0, dtype=float) + eps))
    tprs.append(np.sum((y_val_logit == 1) * (y_preds_val == 1)) / (np.sum(y_val_logit == 1, dtype=float) + eps))

plt.plot(fprs,tprs)
plt.title("ROC curve for Logistic Regression")
plt.ylabel("true positive rate")
plt.xlabel("false positive rate")
plt.show()

In [ ]:
x_prev = 0.000
auc = 0
for x, y in zip(sorted(fprs), sorted(tprs)):
    auc += (x - x_prev)*y
    x_prev = x
print "area under curve: ", auc

In [ ]:
senses = np.array(tprs, dtype=float) + eps
specs = 1 - np.array(fprs, dtype=float) + eps
h_mean = 2. / (1 / senses + 1 / specs)
opt_index = np.argmax(h_mean)
opt_threshold = opt_index*step_size

print "best threshold: " , opt_threshold
print "highest sensitivity: ", tprs[opt_index]
print "highest specificity: ", 1 - fprs[opt_index]

In [ ]:
y_probs_train = lr.predict_proba(x_train)[:,1]
y_preds_train = y_probs_train > opt_threshold

y_probs_test = lr.predict_proba(x_test)[:,1]
y_preds_test = y_probs_test > opt_threshold
print y_probs_test[:5]
print y_preds_test[:5]

In [ ]:
# fit on training data to see if there is underfitting
print 'Training set'
print 'predicted proportion of defaults: ', np.sum(y_preds_train == 1) / m_train
print 'accuracy: ', np.sum(y_preds_train == y_train_logit) / m_train

# fit on test set to see if there is overfitting
print '\nTest set'
print 'predicted proportion of defaults: ', np.sum(y_preds_test == 1) / m_test
print 'accuracy: ', np.sum(y_preds_test == y_test_logit) / m_test

In [ ]:
# to discuss/redo cells below

In [ ]:
# from sklearn import metrics

# print 'Training set'
# print metrics.classification_report(y_train_logit, y_pred_train)
# print '\nTest set'
# print metrics.classification_report(y_test_logit, y_pred_test)

In [ ]:
# ####### optional ##########
# # if import from sorted data - need to downsample train dataset to be more balanced
# # can call the following function multiple times for decreasing in log term
# # dropping PIF samples with 50% probability

# def drop_half(train):
#     print 'before drop: ', train.shape[0]
#     dropidx = []
#     for idx, val in train['LoanStatus'].iteritems():
#         if val != 'CHGOFF':
#             rand = np.random.rand()
#             if rand < 0.5:
#                 dropidx.append(idx)
#     train = train.drop(dropidx)
#     print 'after drop: ', train.shape[0]
#     return train
    
# train = drop_half(train)
# train = drop_half(train)
# # test = drop_half(test)

# chargeoff_count = (train['LoanStatus'] == "CHGOFF").sum()
# print 'Chargeoffs:', chargeoff_count
# print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(train)

# chargeoff_count = (test['LoanStatus'] == "CHGOFF").sum()
# print 'Chargeoffs:', chargeoff_count
# print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(test)